In [2]:
import numpy as np
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(224),  # Required for CNN input size
    transforms.ToTensor()
])

trainset = CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = CIFAR10(root='./data', train=False, download=True, transform=transform)



Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch
import torchvision.models as models
from torch.utils.data import DataLoader

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)
model.fc = torch.nn.Identity()  # remove last classification layer
model.eval()

def extract_features(dataset):
    loader = DataLoader(dataset, batch_size=64, shuffle=False)
    features = []
    labels = []
    with torch.no_grad():
        for imgs, lbls in loader:
            feats = model(imgs).cpu().numpy()  # <-- MOVED TO CPU FIRST
            features.append(feats)
            labels.append(lbls.numpy())
    return np.vstack(features), np.hstack(labels)

x_train, y_train = extract_features(trainset)
x_test, y_test = extract_features(testset)

print(f"Feature shapes: train {x_train.shape}, test {x_test.shape}")


Feature shapes: train (50000, 512), test (10000, 512)


In [5]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Step 1: Scaling on original CNN features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Step 2: PCA
pca_components = 10
pca = PCA(n_components=pca_components, random_state=1234)
x_train_pca = pca.fit_transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

print(f"Shape after PCA: train {x_train_pca.shape}, test {x_test_pca.shape}")

# Step 3: Train and evaluate RBF SVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import time

start_time = time.time()

svm_rbf = SVC(kernel='rbf', gamma='scale', random_state=1234)
svm_rbf.fit(x_train_pca, y_train)

# Predictions
y_pred = svm_rbf.predict(x_test_pca)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM RBF Kernel Accuracy after PCA: {accuracy:.4f}\n")

# Classification report
print("Classification Report after PCA:")
print(classification_report(y_test, y_pred))

print(f"Elapsed time: {time.time() - start_time:.4f} seconds")


Shape after PCA: train (50000, 10), test (10000, 10)
SVM RBF Kernel Accuracy after PCA: 0.6285

Classification Report after PCA:
              precision    recall  f1-score   support

           0       0.71      0.71      0.71      1000
           1       0.70      0.68      0.69      1000
           2       0.52      0.43      0.47      1000
           3       0.45      0.48      0.46      1000
           4       0.57      0.58      0.58      1000
           5       0.63      0.57      0.60      1000
           6       0.65      0.73      0.69      1000
           7       0.69      0.64      0.66      1000
           8       0.67      0.74      0.71      1000
           9       0.67      0.71      0.69      1000

    accuracy                           0.63     10000
   macro avg       0.63      0.63      0.63     10000
weighted avg       0.63      0.63      0.63     10000

Elapsed time: 25.5733 seconds


In [ ]:
import time
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from sklearn.metrics import accuracy_score, classification_report

# Timer
start_time = time.time()

# Set number of qubits (features)
num_features = x_train_pca.shape[1]  # PCA-reduced dimensions (20)

# ZZFeatureMap (pairwise entanglement, 1 rep)
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1, entanglement='circular')

# FidelityQuantumKernel setup with Sampler and ComputeUncompute (matching your setup)
sampler = Sampler()
fidelity = ComputeUncompute(sampler=sampler)
quantum_kernel_gpu = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

# QSVC classifier using quantum kernel
qsvc = QSVC(quantum_kernel=quantum_kernel_gpu)

# Fit QSVM model on PCA training data
qsvc.fit(x_train_pca, y_train)

# Predict with QSVM model on PCA test data
y_pred = qsvc.predict(x_test_pca)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"QSVM (ZZFeatureMap pairwise, 1 rep) Accuracy: {accuracy:.4f}\n")

# Detailed classification report
print("Classification Report (QSVM):")
print(classification_report(y_test, y_pred))

# Elapsed time
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")


/tmp/ipykernel_475/3391722220.py:19: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


In [6]:
x_train_pca.shape[1]


10